In [14]:
import pandas as pd
from IPython.display import Markdown as md

In [2]:
# read data into DataFrame
data = pd.read_csv('PGA_Raw_Data_2017-2022.csv')
data = data.drop(['hole_DKP', 'hole_FDP', 'hole_SDP', 
                  'streak_DKP', 'streak_FDP', 'streak_SDP', 
                  'n_rounds', 'finish_DKP', 'finish_FDP', 
                  'finish_SDP', 'total_DKP', 'total_FDP', 
                  'total_SDP', 'Unnamed: 2', 'Unnamed: 3', 
                  'Unnamed: 4', 'purse', 'no_cut'],
                 axis=1
                );  # the semicolon supresses output, w/o it the df would be printed here

In [3]:
# take a look at the data
data.head(3)

,Player_initial_last,tournament id,player id,hole_par,strokes,made_cut,pos,player,tournament name,course,date,season,Finish,sg_putt,sg_arg,sg_app,sg_ott,sg_t2g,sg_total
0,A. Ancer,401353224,9261,288,289,1,32.0,Abraham Ancer,The Memorial Tournament pres. by Nationwide,"Muirfield Village Golf Club - Dublin, OH",2022-06-05,2022,T32,0.20,-0.13,-0.08,0.86,0.65,0.85
1,A. Hadwin,401353224,5548,288,286,1,18.0,Adam Hadwin,The Memorial Tournament pres. by Nationwide,"Muirfield Village Golf Club - Dublin, OH",2022-06-05,2022,T18,0.36,0.75,0.31,0.18,1.24,1.60
2,A. Lahiri,401353224,4989,144,147,0,NaN,Anirban Lahiri,The Memorial Tournament pres. by Nationwide,"Muirfield Village Golf Club - Dublin, OH",2022-06-05,2022,CUT,-0.56,0.74,-1.09,0.37,0.02,-0.54


## Data Explanation

**Player_initial_last**  
Initial of the players first name and their full last name.  
**tournament id**  
Unique id for each tournament.  
**player id**  
Unique id for each player.  
**hole_par**  
Number of strokes to score a par overall.  
**strokes**  
Number of strokes the player needed for the tournament.  
**made_cut**  
0: did not make cut  
1: made the cut  
**pos**  
Position of the player in the tournament.  
**player**  
The player's full name.  
**tournament name**
Name of the tournament.  
**course**  
Golf course on which was played.  
**date**  
Date of the tournament, format YYYY-MM-DD  
**season**  
Year of the season (2017 - 2022). Keep in mind that season 2017 started in 2016 and so on.  
**Finish**  
Finishing position of the player, if the player was cut the 'Finish' is 'CUT'.  
**sg_put**  
Shots gained while putting.  
**sg_arg**  
Shots gained around the green.  
**sg_app**  
Shots gained during approach.  
**sg_ott**  
Shots gained off the tee.  
**sg_t2g**  
Shots gained tee to green.  
**sg_total**  
Shots gained in total.

In [4]:
data.describe()

,tournament id,player id,hole_par,strokes,made_cut,pos,season,sg_putt,sg_arg,sg_app,sg_ott,sg_t2g,sg_total
count,2.852500e+04,2.852500e+04,28525.000000,28525.000000,28525.000000,16167.000000,28525.000000,23523.000000,23523.000000,23523.000000,23523.000000,23523.000000,23524.000000
mean,3.013480e+08,1.022799e+05,224.712428,223.119755,0.598422,33.883714,2019.428466,-0.125924,-0.044617,-0.108220,-0.049563,-0.202325,-0.323437
std,1.734438e+08,6.523901e+05,70.446659,66.811562,0.490226,22.518739,1.662580,1.121120,0.728476,1.117115,0.803879,1.638853,1.972278
min,2.689000e+03,5.000000e+00,70.000000,66.000000,0.000000,1.000000,2017.000000,-5.990000,-5.410000,-7.510000,-7.740000,-13.950000,-13.670000
25%,4.010252e+08,1.264000e+03,143.000000,146.000000,0.000000,15.000000,2018.000000,-0.780000,-0.450000,-0.750000,-0.460000,-1.100000,-1.410000
50%,4.010566e+08,4.449000e+03,280.000000,271.000000,1.000000,32.000000,2019.000000,-0.040000,-0.010000,-0.010000,0.040000,-0.030000,-0.180000
75%,4.012430e+08,7.001000e+03,287.000000,281.000000,1.000000,51.000000,2021.000000,0.620000,0.410000,0.635000,0.480000,0.910000,1.040000
max,4.013669e+08,4.845309e+06,292.000000,313.000000,1.000000,999.000000,2022.000000,4.300000,3.170000,4.140000,2.470000,5.570000,7.010000


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28525 entries, 0 to 28524
Data columns (total 19 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Player_initial_last  28525 non-null  object 
 1   tournament id        28525 non-null  int64  
 2   player id            28525 non-null  int64  
 3   hole_par             28525 non-null  int64  
 4   strokes              28525 non-null  int64  
 5   made_cut             28525 non-null  int64  
 6   pos                  16167 non-null  float64
 7   player               28525 non-null  object 
 8   tournament name      28525 non-null  object 
 9   course               28525 non-null  object 
 10  date                 28525 non-null  object 
 11  season               28525 non-null  int64  
 12  Finish               23524 non-null  object 
 13  sg_putt              23523 non-null  float64
 14  sg_arg               23523 non-null  float64
 15  sg_app               23523 non-null 

In [9]:
data.isna()

,Player_initial_last,tournament id,player id,hole_par,strokes,made_cut,pos,player,tournament name,course,date,season,Finish,sg_putt,sg_arg,sg_app,sg_ott,sg_t2g,sg_total
0,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28520,False,False,False,False,False,False,False,False,False,False,False,False,True,True,True,True,True,True,True
28521,False,False,False,False,False,False,False,False,False,False,False,False,True,True,True,True,True,True,True
28522,False,False,False,False,False,False,True,False,False,False,False,False,True,True,True,True,True,True,True
28523,False,False,False,False,False,False,False,False,False,False,False,False,True,True,True,True,True,True,True


## Taking a look at Justin Thomas

In [10]:
# store data regarding Justin Thomas in jt_data
jt_data = data[data['player']=='Justin Thomas']

In [11]:
jt_data.describe()

,tournament id,player id,hole_par,strokes,made_cut,pos,season,sg_putt,sg_arg,sg_app,sg_ott,sg_t2g,sg_total
count,1.190000e+02,119.0,119.000000,119.000000,119.000000,104.000000,119.000000,91.000000,91.000000,91.000000,91.000000,91.000000,91.000000
mean,2.932912e+08,4848.0,264.932773,257.142857,0.865546,14.173077,2019.294118,0.028571,0.295824,0.769121,0.353187,1.418242,1.460659
std,1.786250e+08,0.0,49.258008,45.228850,0.342582,14.171928,1.679149,0.997236,0.503799,0.938631,0.538896,1.110451,1.618174
min,2.689000e+03,4848.0,140.000000,141.000000,0.000000,1.000000,2017.000000,-2.910000,-1.350000,-2.410000,-1.470000,-3.060000,-3.940000
25%,3.790000e+03,4848.0,280.000000,268.000000,1.000000,3.000000,2018.000000,-0.510000,-0.050000,0.235000,0.050000,0.955000,0.725000
50%,4.010565e+08,4848.0,284.000000,274.000000,1.000000,9.000000,2019.000000,0.190000,0.360000,0.880000,0.300000,1.530000,1.670000
75%,4.012334e+08,4848.0,288.000000,279.000000,1.000000,21.000000,2021.000000,0.765000,0.645000,1.450000,0.710000,2.035000,2.465000
max,4.013669e+08,4848.0,292.000000,292.000000,1.000000,75.000000,2022.000000,1.810000,1.690000,2.700000,1.750000,3.610000,4.380000


In [24]:
jt_cut = jt_data[jt_data['made_cut']==0]
jt_cut = jt_cut['made_cut'].count()  # number of times JT has been cut

jt_not_cut = jt_data[jt_data['made_cut']==1]
jt_not_cut = jt_not_cut['made_cut'].count()  # number of times JT made the cut

print(f'jt_not_cut: {jt_not_cut}\njt_cut: {jt_cut}')

jt_not_cut: 103
jt_cut: 16


#### Cuts made by Justin Thomas
Justin Thomas made 103 cuts out of 119 between 2017 and 2022.

#### In which part of his game does Justin Thomas gain the most or least shots?

In [38]:
# tee to green and total can be ignored here, because they would include (almost) every part of the game
jt_avg_sg_putt = jt_data['sg_putt'].mean()  # mean shots gained putting
jt_avg_sg_arg = jt_data['sg_arg'].mean()  # mean shots gained around the green
jt_avg_sg_app = jt_data['sg_app'].mean()  # mean shots gained on approach
jt_avg_sg_ott = jt_data['sg_ott'].mean()  # mean shots gained off the tee

# dict with sg
jt_dict_mean_sg = {'Putting':jt_avg_sg_putt, 
                   'Around the Green':jt_avg_sg_arg, 
                   'Approach':jt_avg_sg_app, 
                   'Off the Tee':jt_avg_sg_ott}

# get variable with max and min value
jt_max_sg_var = max(jt_dict_mean_sg, key=jt_dict_mean_sg.get)
jt_min_sg_var = min(jt_dict_mean_sg, key=jt_dict_mean_sg.get)

print(f'Mean:\nSG Putting: {jt_avg_sg_putt}\nSG ARG: {jt_avg_sg_arg}\nSG App: {jt_avg_sg_app}\nSG OTT: {jt_avg_sg_ott}')
print(f'Most SG: {jt_max_sg_var}\nLeast SG: {jt_min_sg_var}')

Mean:
SG Putting: 0.028571428571428557
SG ARG: 0.29582417582417575
SG App: 0.7691208791208792
SG OTT: 0.3531868131868132
Most SG: Approach
Least SG: Putting


Justin Thomas has the highest amount of shots gained during his approach while he struggles with gaining shots whilst putting.